In [1]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
import os
import sys
# Needed for the model class to work inside a Notebook.
os.environ["COBAYA_NOMPI"] = "True"

from cobaya.yaml          import yaml_load_file
from cobaya.samplers.mcmc import plot_progress
from cobaya.model         import get_model
from cobaya.samplers.mcmc import plot_progress
#
from getdist.mcsamples    import MCSamplesFromCobaya
from getdist.mcsamples    import loadMCSamples
import getdist.plots      as     gdplt
#
import os

In [3]:
# Add everything to our paths.
repo = "/global/cscratch1/sd/sfschen/CobayaBAO_DESI/"
for codedir in ["lss_likelihood"]:
    sys.path.append(repo+codedir)

In [44]:
nm = 81
samp = 'z%2d'%(nm)
#info = yaml_load_file("recsym_z%2d_pkells_r10_kmin_02_kmax30.yaml"%(nm))
info = yaml_load_file("recsym_z%2d_pkells.yaml"%(nm))
print(info['params'].keys())
print(info['output'])

dict_keys(['apar', 'aperp', 'B1_z81', 'F_z81', 'M0_z81', 'M1_z81', 'M2_z81', 'M3_z81', 'M4_z81', 'Q0_z81', 'Q1_z81', 'Q2_z81', 'Q3_z81', 'Q4_z81'])
chains/recsym_z81_pkells


In [45]:
params = {'apar': 0.99875656, 'aperp': 1.0082069,\
          'B1_z%2d'%(nm): 0.87570316, 'F_z%2d'%(nm): 0.77598726,\
          'M0_z%2d'%(nm): 233.14934, 'M1_z%2d'%(nm): 675.8675, 'M2_z%2d'%(nm): -462.8388, 'M3_z%2d'%(nm): 170.85299999999998, 'M4_z%2d'%(nm): -13.268070999999999,\
          'Q0_z%2d'%(nm): 271.81997, 'Q1_z%2d'%(nm): -351.08239, 'Q2_z%2d'%(nm): 611.80872, 'Q3_z%2d'%(nm): -1287.7481, 'Q4_z%2d'%(nm): 269.95396}

In [46]:
model = get_model(info)
print(list(model.parameterization.sampled_params()))

[model] *WARNING* Ignored blocks/options: ['sampler', 'output', 'stop_on_error']
(10.556307852555234, 9.45499094916356, 10.565115328400125, 13.777685164072894, 0.03123000341125935)
['apar', 'aperp', 'B1_z81', 'F_z81', 'M0_z81', 'M1_z81', 'M2_z81', 'M3_z81', 'M4_z81', 'Q0_z81', 'Q1_z81', 'Q2_z81', 'Q3_z81', 'Q4_z81']


In [47]:
model.logposterior(params)
lik = model.likelihood['recsym_likelihood.RecSymLikelihood']

tt = lik.bao_predict(samp)
tt = lik.bao_observe(tt)

In [48]:
# Compute templates:
lik_str = 'recsym_likelihood.RecSymLikelihood'
pars_list = params.keys()

In [49]:
err = 0.005
dPs = {}

for param_str in pars_list:
    param = params[param_str]
    
    pars_plus = params.copy()
    pars_plus[param_str] = param + err
    
    pars_minus = params.copy()
    pars_minus[param_str] = param - err

    model.logposterior(pars_plus)
    obs_plus = 1.0 * model.likelihood[lik_str].thy_obs
        
    model.logposterior(pars_minus)
    obs_minus = 1.0 * model.likelihood[lik_str].thy_obs
        
    dPs[param_str] = (obs_plus - obs_minus)/(2*err)

In [50]:
# Dot into covariance to make Fisher matrix
Fij = np.zeros( (len(params.keys()),)*2 )
    
for ii, param_ii in enumerate(pars_list):
    for jj, param_jj in enumerate(pars_list):
            
        Fij[ii,jj] = np.dot(dPs[param_ii],\
                                np.dot(model.likelihood[lik_str].cinv,\
                                       dPs[param_jj]))

In [51]:
np.diag(np.sqrt(np.linalg.inv(Fij)))

<ipython-input-51-7dce64f41c76>:1: RuntimeWarning: invalid value encountered in sqrt
  np.diag(np.sqrt(np.linalg.inv(Fij)))


array([1.54862918e-02, 9.39992135e-03, 9.40221492e-02, 2.04961544e-01,
       4.13171159e+01, 1.01392743e+03, 6.13254680e+02, 2.94554247e+02,
       5.03378868e+01, 1.37985792e+02, 2.39724698e+03, 1.41452488e+03,
       6.49758541e+02, 1.09674091e+02])

In [52]:
# Add Priors
iis =  (np.arange(len(pars_list) ) ) > 3 # no apar, aperp, B1, F
prior_inv_sigmas = iis * 1./100000**2

Fij_with_prior = Fij + np.diag(prior_inv_sigmas)

In [53]:
np.diag(np.sqrt(np.linalg.inv(Fij_with_prior)))

<ipython-input-53-857f7eb08c5c>:1: RuntimeWarning: invalid value encountered in sqrt
  np.diag(np.sqrt(np.linalg.inv(Fij_with_prior)))


array([1.54859628e-02, 9.39972555e-03, 9.40062983e-02, 2.04903845e-01,
       4.13149432e+01, 1.01385959e+03, 6.13222332e+02, 2.94546524e+02,
       5.03369360e+01, 1.37952546e+02, 2.39642651e+03, 1.41415129e+03,
       6.49679612e+02, 1.09665072e+02])

In [54]:
np.linalg.inv(Fij_with_prior)[:4,:4]

array([[ 2.39815044e-04, -4.82604120e-05,  1.84385486e-04,
        -6.92550534e-04],
       [-4.82604120e-05,  8.83548404e-05, -2.46580876e-04,
         3.73954092e-04],
       [ 1.84385486e-04, -2.46580876e-04,  8.83718412e-03,
        -1.41901768e-02],
       [-6.92550534e-04,  3.73954092e-04, -1.41901768e-02,
         4.19855859e-02]])

In [55]:
corr_mat = np.linalg.inv(Fij_with_prior)/np.diag(np.sqrt(np.linalg.inv(Fij_with_prior)))[:,None]/np.diag(np.sqrt(np.linalg.inv(Fij_with_prior)))[None,:]

<ipython-input-55-9a6d4c649bc8>:1: RuntimeWarning: invalid value encountered in sqrt
  corr_mat = np.linalg.inv(Fij_with_prior)/np.diag(np.sqrt(np.linalg.inv(Fij_with_prior)))[:,None]/np.diag(np.sqrt(np.linalg.inv(Fij_with_prior)))[None,:]


In [56]:
corr_mat[:4,:4]

array([[ 1.        , -0.3315413 ,  0.12665769, -0.21825447],
       [-0.3315413 ,  1.        , -0.27905337,  0.19415698],
       [ 0.12665769, -0.27905337,  1.        , -0.73668317],
       [-0.21825447,  0.19415698, -0.73668317,  1.        ]])

In [17]:
np.diag(np.sqrt(np.linalg.inv(Fij_with_prior[:4,:4])))

<ipython-input-17-6b31f6b9f5ce>:1: RuntimeWarning: invalid value encountered in sqrt
  np.diag(np.sqrt(np.linalg.inv(Fij_with_prior[:4,:4])))


array([0.01297143, 0.00856761, 0.01795289, 0.03575121])

In [18]:
np.diag(np.sqrt(np.linalg.inv(Fij_with_prior[:2,:2])))

<ipython-input-18-2c22f3d3ae50>:1: RuntimeWarning: invalid value encountered in sqrt
  np.diag(np.sqrt(np.linalg.inv(Fij_with_prior[:2,:2])))


array([0.00429801, 0.0028842 ])

In [19]:
corr_mat = np.linalg.inv(Fij_with_prior[:4,:4])/np.diag(np.sqrt(np.linalg.inv(Fij_with_prior[:4,:4])))[:,None]/np.diag(np.sqrt(np.linalg.inv(Fij_with_prior[:4,:4])))[None,:]

<ipython-input-19-800a0b0b04ac>:1: RuntimeWarning: invalid value encountered in sqrt
  corr_mat = np.linalg.inv(Fij_with_prior[:4,:4])/np.diag(np.sqrt(np.linalg.inv(Fij_with_prior[:4,:4])))[:,None]/np.diag(np.sqrt(np.linalg.inv(Fij_with_prior[:4,:4])))[None,:]


In [20]:
corr_mat[:4,:4]

array([[ 1.        , -0.36600074,  0.48918734, -0.90307345],
       [-0.36600074,  1.        , -0.93029699,  0.59470038],
       [ 0.48918734, -0.93029699,  1.        , -0.74390488],
       [-0.90307345,  0.59470038, -0.74390488,  1.        ]])